# simrank原生矩阵算法

In [7]:
import numpy as np
import time

## 数据预处理

In [10]:
def getGraph(edge_num):
    with open('test/web-Stanford.txt') as log_fp:
        logs = []
        for i in range(edge_num):
            log = log_fp.readline()
            logs.append(log.strip())    #strip是去掉空格和换行
    
    #from to元祖
    logs_tuple = [ tuple(log.split('\t')) for log in logs ]
    
    #获取from集合和to集合
    from_ = list(set([ log[0] for log in logs_tuple ]))
    to_ = list(set( [log[1] for log in logs_tuple ]))
    
    #顶点集合
    vertex = list(set(from_ + to_))
    
    #图结构邻接矩阵表示
    #图的所有节点都放进去
    #即A矩阵
    graph = np.matrix(np.zeros([len(vertex), len(vertex)]))

    #给图邻接矩阵添加边
    #有向图，非对称
    for log in logs_tuple:
        i = vertex.index(log[0])
        j = vertex.index(log[1])
        graph[i, j] = 1   #行为出度，列为入度
        
    return graph, vertex

## simrank

In [11]:
graph, vertex = getGraph(10)

In [12]:
print(len(vertex))
graph

12


matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.]])

In [ ]:
#根据A矩阵构造转移矩阵P
for i in range(len(vertex)):
    #每列之和
    num = sum(graph[:, i])
    #该列每个元素除以和，得到归一化，和为1
    for j in range(len(vertex)):
        if num != 0:
            graph[j, i] = graph[j, i] / num
            
graph

In [47]:
#相似度矩阵S
S = np.matrix(np.identity(len(vertex)))
S

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]])

In [37]:
#计算simrank
def simrank(C = 0.8, K = 5):
    global S
    
    #单位矩阵
    I = np.matrix(np.identity(len(vertex)))
    for i in range(K):
        temp = C * graph.T * S * graph
        S = temp + I - np.diag(np.diag(temp))

In [48]:
start = time.process_time()
simrank()
end = time.process_time()
print(end - start)
S

3.3072212000000007


matrix([[1. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 1. , 0.8, ..., 0. , 0. , 0. ],
        [0. , 0.8, 1. , ..., 0. , 0. , 0. ],
        ...,
        [0. , 0. , 0. , ..., 1. , 0. , 0.8],
        [0. , 0. , 0. , ..., 0. , 1. , 0. ],
        [0. , 0. , 0. , ..., 0.8, 0. , 1. ]])